## Loading the data

* since we have directory, lets use directory loader

In [1]:
from langchain_community.document_loaders import DirectoryLoader

In [2]:
direcory_loader = DirectoryLoader(
    path="./data",
    glob="**/*.txt",
    show_progress=True,
    use_multithreading=True)

raw_documents = direcory_loader.load()

  0%|          | 0/116 [00:00<?, ?it/s]libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
libmagic is unavailable but assists in filetype detection. Please consider installing libmagic for better results.
Need to load profiles.
Need to load profiles.
Need to load profiles.
short text: "Title: Understanding CI/CD Pipelines". Defaulting to English.
short text: "Title: Introduction to Terraform". Defaulting to English.
short text: "Title: Kubernetes Basics". Defaulting to English.
Need to load profiles.
Need to load profiles.
Need to load profiles.
Need to load profiles.
  1%|          | 1/116 [00:07<14:53,  7.77s/it]Need to load profiles.
Need to load profiles.
  2%|▏         | 2/116 [00:07<09:18,  

In [3]:
len(raw_documents)

116

In [4]:
print(raw_documents[0].page_content)

Title: Understanding CI/CD Pipelines

Overview: CI/CD (Continuous Integration and Continuous Deployment) is a process that enables software teams to deliver code changes frequently and reliably.

Key Concepts: 1. Continuous Integration – Developers merge code into a shared repository frequently. 2. Continuous Deployment – Automated deployment of tested code into production. 3. Benefits – Faster delivery, fewer bugs, and improved collaboration.


# We have documents but we need to chunk
* Since the nature of data is text which has paragraphs, lines etc
* [Splitters](https://python.langchain.com/docs/concepts/text_splitters/)

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 200,
    chunk_overlap=10,
    separators=["\n", "\n\n"],
)
raw_documents_post_split = text_splitter.split_documents(raw_documents)

In [6]:
print(f"raw_documents before split {len(raw_documents)}")
print(f"raw_documents post split {len(raw_documents_post_split)}")

raw_documents before split 116
raw_documents post split 948


In [7]:
print(raw_documents_post_split[0].page_content)


Title: Understanding CI/CD Pipelines

Overview: CI/CD (Continuous Integration and Continuous Deployment) is a process that enables software teams to deliver code changes frequently and reliably.


## We need to choose an embedding model and vector store.

* Lets use [Text embedding from gcp](https://cloud.google.com/vertex-ai/generative-ai/docs/embeddings/get-text-embeddings#get-text-embeddings-for-a-snippet-of-text)
* Vector store, lets use chromadb



In [8]:
from langchain_google_vertexai import VertexAIEmbeddings

embeddings = VertexAIEmbeddings(model="text-embedding-005")

c:\khajaclassroom\GenerativeAI\agenticai\aug25\kb_articles_rag\.venv\Lib\site-packages\vertexai\_model_garden\_model_garden_models.py:278: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


In [9]:
from langchain_chroma import Chroma

In [10]:
vector_store = Chroma(
    collection_name="kb_collection",
    embedding_function=embeddings,
    persist_directory="./vectordb",
)

In [11]:
# Now add documents
vector_store.add_documents(raw_documents_post_split)

['9e35bf5c-8385-4fc8-b7c4-df92c1b389ba',
 'f2298f82-ccca-4927-865c-d163852a42fd',
 '27758081-3df2-4b4e-92f5-6766312158da',
 '7f16062a-53b2-4368-905c-0a2bea125479',
 'c497b215-e4fa-459a-8355-d4ec41b69416',
 'a551a379-6193-4306-bba9-56d44dac7c17',
 '806939f2-ca15-41d5-8015-23455d84d43b',
 '62220e96-05d6-4816-b7c0-7b5aa4cf7026',
 '9810c07a-7297-4fbe-8be5-a3dbb8bf5870',
 'f8e3b257-66ee-4928-8924-2142da93a088',
 'ca24ed44-475e-43df-aa56-01503e06ad32',
 'd3800920-a69f-4f89-84a3-fbf4df43a6c6',
 '217ac87f-c4f5-42e7-ad69-b7a018ffcbee',
 '4b9153f8-170d-456e-b598-37ed90cc2389',
 '202ccb5a-4352-4db5-8242-c2e3d48a0848',
 '889fa436-c105-4819-a3af-3a09b4f21465',
 'f8bab9be-a263-4a3d-bc9a-a5a961bc86fb',
 '1f019e2c-12c1-4a7f-865e-98b5d937ae19',
 '0d578828-74fe-48e5-b63d-d7c60a295cec',
 '70643214-234b-47b1-9745-be7143df7f64',
 '8cd04abe-53a5-4a9d-b68f-9e475e6d6974',
 '2f0bfd35-dd80-4d2f-aded-a02b59d5ca7c',
 'c6ef8b29-c201-4fe1-9100-5a90e2dc0bc0',
 '2bf0332b-ff35-4dfd-8298-5cc41477be0a',
 '529e7e99-2f08-

In [12]:
# lets experiment
retriever = vector_store.as_retriever()

In [13]:
results = retriever.invoke("what are CI/CD Pipelines ?")

In [14]:
len(results)

4

In [15]:
for result in results:
    print(result.metadata)

{'source': 'data\\article_001_ci_cd_pipeline.txt'}
{'source': 'data\\article_001_ci_cd_pipeline.txt'}
{'source': 'data\\article_048_ci_cd_tools.txt'}
{'source': 'data\\article_048_ci_cd_tools.txt'}


In [16]:
for result in results:
    print(result.page_content)

Title: Understanding CI/CD Pipelines

Overview: CI/CD (Continuous Integration and Continuous Deployment) is a process that enables software teams to deliver code changes frequently and reliably.
Title: Understanding CI/CD Pipelines

Overview: CI/CD (Continuous Integration and Continuous Deployment) is a process that enables software teams to deliver code changes frequently and reliably.
Title: Popular CI/CD Tools

Overview: Several tools help implement CI/CD pipelines.
Title: Popular CI/CD Tools

Overview: Several tools help implement CI/CD pipelines.


In [17]:
# few shot prompting
from langchain.chat_models import init_chat_model
llm = init_chat_model("gemini-2.5-flash-lite", model_provider="google_vertexai")

In [18]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
messages = [
    SystemMessage(content="You are a senior in college who has knowledge of computer science and responds in oneliners"),
    HumanMessage(content="What are graphs?"),
    AIMessage(content="Graphs are part of datastructures"),
    HumanMessage(content="What are trees in computer science?"),
    AIMessage(content="Trees are part of datastructures"),
    HumanMessage(content="What are compilers ?")
]

In [19]:
response = llm.invoke(input=messages)

In [20]:
type(response)

langchain_core.messages.ai.AIMessage

In [21]:
response.pretty_print()

================================== Ai Message ==================================

Compilers translate source code into machine code.


In [22]:
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
messages = [
    SystemMessage(content="You are a patient school teacher"),
    HumanMessage(content="What is addition ?"),
    AIMessage(content="If you have 10 rupees and your father gives you 5 rupees, what the total amount, count it answer shoule be 15"),
    HumanMessage(content="What is subtraction ?"),
    AIMessage(content="You have 10 chocolates, you ate 2 how many are left, count it it should be 8"),
    HumanMessage(content="What is division ?")
]

In [23]:
response = llm.invoke(messages)
response.pretty_print()

================================== Ai Message ==================================

Imagine you have 12 delicious cookies, and you want to share them equally among your 3 best friends. Division is like figuring out how many cookies each friend gets so that everyone has the same amount.

So, if you have 12 cookies and 3 friends, you'd divide 12 by 3.

Can you tell me how many cookies each friend would get? 🤔


In [24]:
prompt_template_string = """ You are an helpful assistant trying to help employees understand the kb articles

Question: {question}

Context: {context}

Using the context give a simple answer

"""

In [25]:
question = input("What are you searching for ?")
retriever = vector_store.as_retriever()
response = retriever.invoke(question)

InvalidArgument: 400 The text content is empty.

In [ ]:
response

[Document(id='eebf5e81-3fb2-4ccb-a13f-bed9d402e67d', metadata={'source': 'data\\article_033_api_gateway.txt'}, page_content='Title: Role of API Gateways\n\nOverview: An API Gateway manages API requests between clients and services.'),
 Document(id='f084d311-0d1a-439b-8deb-c8cbfff65f7d', metadata={'source': 'data\\article_005_api_design.txt'}, page_content='Title: Best Practices for API Design\n\nOverview: APIs should be intuitive, consistent, and easy to consume.'),
 Document(id='fc5365bf-4d9a-4243-96d2-b39a3f9d19de', metadata={'source': 'data\\article_057_api_testing.txt'}, page_content='Title: API Testing Approaches\n\nOverview: API testing validates endpoints for functionality and performance.\n\nApproaches:\n\n1. Functional testing.\n\n2. Load testing.\n\n3. Security testing.'),
 Document(id='b57b5c1a-c723-4a46-8d7e-6062640c7a59', metadata={'source': 'data\\article_005_api_design.txt'}, page_content='Best Practices: 1. Use RESTful principles or GraphQL when suitable. 2. Version API

In [ ]:
context = ""
for doc in response:
    context = f"{context}\n{doc.page_content}"
print(context)



Title: Role of API Gateways

Overview: An API Gateway manages API requests between clients and services.
Title: Best Practices for API Design

Overview: APIs should be intuitive, consistent, and easy to consume.
Title: API Testing Approaches

Overview: API testing validates endpoints for functionality and performance.

Approaches:

1. Functional testing.

2. Load testing.

3. Security testing.
Best Practices: 1. Use RESTful principles or GraphQL when suitable. 2. Version APIs properly. 3. Provide clear documentation.


In [ ]:
from langchain_core.prompts import PromptTemplate
template = PromptTemplate.from_template(prompt_template_string)

In [ ]:
chain = template | llm

In [ ]:
# RAG's reponse
response = chain.invoke({'question': question, 'context': context})
response.pretty_print()

================================== Ai Message ==================================

Based on the provided context, an API (Application Programming Interface) is a way for different services or software to communicate with each other. Think of it as a messenger that handles requests between a client (like a user's device) and the services it needs information from.

The articles highlight that APIs should be designed to be easy to use, managed by gateways, and tested for their functionality, performance, and security.


In [ ]:
# models response
llm.invoke(question).pretty_print()

================================== Ai Message ==================================

API stands for **Application Programming Interface**.

In its simplest terms, an API is a **set of rules, protocols, and tools that allows different software applications to communicate and interact with each other.** Think of it as a **messenger** or a **contract** that defines how one piece of software can request services or data from another.

Here's a breakdown of the key concepts:

**1. Interface:**

* **It's a boundary:** Just like a physical interface (like a USB port) allows different devices to connect, an API provides a defined way for software components to connect and exchange information.
* **It's what you see and interact with:** You don't need to know the intricate internal workings of another application to use its API. You only need to understand the API's specifications.

**2. Programming:**

* **It's for software developers:** APIs are designed for programmers to use when building new 

In [ ]:
Prom